In [1]:
from langchain_core.pydantic_v1 import BaseModel, Field

class InterviewTemplate(BaseModel):
    pass

class InterviewSubject(InterviewTemplate):
    first_name: str = Field(
        ...,
        description="The first name of the interview subject",
        examples=["John", "Jane", "Alice"],
    )
    last_name: str = Field(
        ...,
        description="The last name of the interview subject",
        examples=["Doe", "Smith", "Johnson"],
    )
    full_name: str = Field(
        ...,
        description="The full name of the interview subject",
        examples=["John Doe", "Jane Smith", "Alice Johnson"],
    )
    supervisor_first_name: str = Field(
        ...,
        description="The first name of the supervisor",
        examples=["John", "Jane", "Alice"],
    )
    supervisor_last_name: str = Field(
        ...,
        description="The last name of the supervisor",
        examples=["Doe", "Smith", "Johnson"],
    )
    supervisor_full_name: str = Field(
        ...,
        description="The full name of the supervisor",
        examples=["John Doe", "Jane Smith", "Alice Johnson"],
    )
    last_day_of_work: str = Field(
        ...,
        description="The last day of work for the interview subject",
    )
    department: str = Field(
        ...,
        description="The department the interview subject worked in",
    )
    role: str = Field(
        ...,
        description="The role of the interview subject",
    )

In [2]:
from langchain_ollama import ChatOllama

model = llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)
structured_llm = model.with_structured_output(InterviewSubject)

In [3]:
user_input = """
    Hi, I'm Paul.
    My last day of work is 2022-12-31.
    I worked in the IT department as a software engineer.
    My supervisor is John Doe.
    """

In [4]:
import json

subject_information = structured_llm.invoke(user_input)
print(json.dumps(subject_information.dict(), indent=4))

{
    "first_name": "Paul",
    "last_name": "",
    "full_name": "Paul",
    "supervisor_first_name": "John",
    "supervisor_last_name": "Doe",
    "supervisor_full_name": "John Doe",
    "last_day_of_work": "2022-12-31",
    "department": "IT",
    "role": "software engineer"
}


In [5]:
def check_what_is_empty(details):
    ask_for = []
    # Check if fields are empty
    for field, value in details.dict().items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            print(f"Field '{field}' is empty.")
            ask_for.append(f'{field}')
    return ask_for

ask_for = check_what_is_empty(details=subject_information)
print(ask_for)

Field 'last_name' is empty.
['last_name']


In [6]:
user_response = """
Sorry about that, Kingston is the name. Paul Kingston.
"""

In [7]:
structured_llm = model.with_structured_output(InterviewSubject)
new_information = structured_llm.invoke(user_response)
print(json.dumps(new_information.dict(), indent=4))

{
    "first_name": "Paul",
    "last_name": "Kingston",
    "full_name": "Paul Kingston",
    "supervisor_first_name": "",
    "supervisor_last_name": "",
    "supervisor_full_name": "",
    "last_day_of_work": "",
    "department": "",
    "role": ""
}


In [8]:
## checking the response and adding it
def add_non_empty_details(current_details: InterviewSubject, new_details: InterviewSubject):
    non_empty_details = {k: v for k, v in new_details.dict().items() if v not in [None, ""]}
    updated_details = current_details.copy(update=non_empty_details)
    return updated_details

current_details = add_non_empty_details(current_details=subject_information, new_details=new_information)
print(json.dumps(current_details.dict(), indent=4))


{
    "first_name": "Paul",
    "last_name": "Kingston",
    "full_name": "Paul Kingston",
    "supervisor_first_name": "John",
    "supervisor_last_name": "Doe",
    "supervisor_full_name": "John Doe",
    "last_day_of_work": "2022-12-31",
    "department": "IT",
    "role": "software engineer"
}


In [10]:
ask_for = check_what_is_empty(details=current_details)
if not ask_for:
    print("All fields are filled.")

All fields are filled.


In [11]:
def check_what_is_empty(current_details: InterviewTemplate):
    ask_for = []
    for field, value in current_details.dict().items():
        if value in [None, "", 0]:
            print(f"Field '{field}' is empty.")
            ask_for.append(f'{field}')
    return ask_for


def add_non_empty_details(current_details: InterviewTemplate, new_details: InterviewTemplate):
    non_empty_details = {k: v for k, v in new_details.dict().items() if v not in [None, ""]}
    updated_details = current_details.copy(update=non_empty_details)
    return updated_details


def filter_response(text_input: str, user_details: InterviewTemplate = None):
    chain = model.with_structured_output(InterviewTemplate)
    result = chain.run(text_input)
    user_details = add_non_empty_details(
        current_details=user_details,
        new_details=result
    )
    ask_for = check_what_is_empty(current_details=user_details)
    return user_details, ask_for